In [1]:
import pandas as pd
import os
import numpy as np
from ensure import ensure_annotations
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
print(os.getcwd())
print(os.chdir(r'd:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem'))
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem\notebook\feature_engineering
None
d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem


In [4]:
df = pd.read_csv(r'data\raw_data\motorola_mobile_data.csv',index_col=None)

In [5]:
df.isnull().sum()

name            0
ratings         0
price           0
imgURL          0
storage_ram     0
camera          0
oS_Processor    0
display         0
network         0
battery         0
dtype: int64

In [6]:
@ensure_annotations
def random_values(feature_name=['price','name','display'],total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[feature_name].iloc[index])
            

In [7]:
df.shape

(120, 10)

## Price

In [11]:
df.reset_index(inplace=True)
for Index,i in enumerate(df['price']):
    i = str(i).replace('₹','')
    i = str(i).replace(',','')
    df['price'][Index] = i

In [13]:
df['price'] = df['price'].astype('Int16')

In [14]:
df['price'] = df['price'].abs() # this is to remove the negative values from price feature

In [15]:
df = df.drop(index=df[df['price'] <=6000].index) # droping the mobiles less than 6000

## Ratings

In [18]:
df['ratings'].dtype

dtype('float64')

## Ram

In [19]:
def extract_ram(row):
    ram = row.split()[2]
    return ram

df['ram'] = df['storage_ram'].apply(func=extract_ram)

In [20]:
import warnings
warnings.filterwarnings('ignore')
df.reset_index(inplace=True)
count = 0
for i in range(0,len(df['ram'])):
    # df['ram'][i]= df['ram'].iloc[i].replace('MBRAM','')
    df['ram'][i]= df['ram'].iloc[i].replace('GBRAM','')
    count+=1
print(f'looped till {count} index')

looped till 110 index


In [21]:
df['ram'] = df['ram'].astype('Int16')

## storage

In [22]:
def extract_storage(row):
    storage = row.split()[1]
    return storage


df['storage'] = df['storage_ram'].apply(func=extract_storage)

In [23]:
df.columns

Index(['level_0', 'index', 'name', 'ratings', 'price', 'imgURL', 'storage_ram',
       'camera', 'oS_Processor', 'display', 'network', 'battery', 'ram',
       'storage'],
      dtype='object')

In [24]:
df = df.drop(columns=['level_0','index'])

In [25]:
import warnings
warnings.filterwarnings('ignore')
df.reset_index(inplace=True)
count = 0
for i in range(0,len(df['storage'])):
    df['storage'][i]= df['storage'].iloc[i].replace('Storage','')
    count+=1
print(f'looped till {count} index')

looped till 110 index


In [26]:
df['storage'] = df['storage'].astype('Int16')
## Removing storage_ram
df = df.drop(columns=['storage_ram','index'])

## Android Version

In [27]:
df.rename({'camera':'os_processor','oS_Processor':'camera'},axis=1,inplace=True)

In [29]:
def android_version(text_):
    split = text_.split()
    if 'OxygenProcessor' in split:
        return 'OxygenProcessor'
    elif 'ANdroid' in split:
        return split[3]
    elif 'OxygenOSProcessor' in split:
        return 'OxygenOSProcessor'
    else:
        pattern=r'^\d+[Processor]|\d+[GProcessor]|\d++[.Processor]'
        found = re.findall(pattern,text_)
        if found:
            return found[0]
        else:
            return 'other androidVerion'

In [30]:
df['android_version'] = df['os_processor'].apply(android_version)

In [31]:
df.reset_index(inplace=True)
for Index,i in enumerate(df['android_version']):
    # print(i)
    i = str(i).replace('P','')
    i = str(i).replace('.','')
    df['android_version'][Index] = i

## Processor

In [45]:
def processor(text):
    pattern=r'(?<=Type).*'
    search = re.search(pattern,text)
    if search != None:
        return ' '.join(re.findall(pattern,text))
    else:
        return 'not montioned'

In [46]:
df['processor'] = df['os_processor'].apply(processor)
df = df.drop('os_processor',axis=1)
df = df.drop('index',axis=1)


In [47]:
def processor_extract(text):
    splitted_sentence = text.split()
    if 'Mediatek' and 'Helio' in splitted_sentence:
        pattern = r'\d+[Processor]|\d+'
        string = ' '.join(splitted_sentence)
        match = re.search(pattern,string)
        if match == None:
            return 'Heilo'+str(splitted_sentence[2])
        else:
            find = re.findall(pattern,string)
            return 'Heilo'+''.join(find)


    elif 'Mediatek' and 'Dimensity' in splitted_sentence:
        pattern = r'(\d+)+[Processor]'
        string = ' '.join(splitted_sentence)
        match = re.search(pattern,string)
        if match == None:
            if 'Dimensity' == str(splitted_sentence[1]):
                return 'Dimensity'+str(splitted_sentence[2])
            else:
                return 'Dimensity'+str(splitted_sentence[1])
        
        else:
            find = re.findall(pattern,string)
            return 'Dimensity'+ ''.join(find)

    elif 'Snapdragon' and 'Gen' and '4' in splitted_sentence:
        return 'Gen4'   

    elif 'Snapdragon' and 'Gen' and '8' in splitted_sentence:         
        return 'Gen8'
    
    elif 'Unisoc' in splitted_sentence:
        
        if 'Unisoc' and 'Tiger' in splitted_sentence:
            pattern = r'[T]\d[Processor]'
            string = ' '.join(splitted_sentence)
            match = re.findall(pattern,string)
            return 'UnisocTiger' + ''.join(splitted_sentence[2])
        
        elif 'Unisoc' == splitted_sentence[0]:
            pattern = r'[T]\d.'
            string = ' '.join(splitted_sentence)
            match = re.findall(pattern,string)
            return 'Unisoc' + ''.join(splitted_sentence[1])

        
        pattern = r'T(\d+)Processor|\d+'
        string = ' '.join(splitted_sentence)
        match = re.findall(pattern,string)
        return 'Unisoc'+' '.join(match)
    

    elif 'Snapdragon' in splitted_sentence:
        pattern = r'\d+[Processor]'
        if 'Gen' and '4' and '8' not in splitted_sentence:
            string = ' '.join(splitted_sentence)
            match = re.search(pattern,string)
            if match == None:
                if 'Snapdragon' == str(splitted_sentence[1]):
                    return 'Snapdragon'+str(splitted_sentence[2])
                else:
                    return 'Snapdragon'+str(splitted_sentence[1])
            else:
                find = re.findall(pattern,string)
                return 'Snapdragon'+''.join(find)
    
    elif 'MediaTek' == splitted_sentence[0]:
        pattern = r'[P]\d.'
        string = ' '.join(splitted_sentence)
        match = re.findall(pattern,string)
        return 'MediaTek' + ' '.join(match)

    else:
        return 'not mentioned'      


In [48]:
df['processor']=df['processor'].apply(processor_extract)

## camera

In [49]:
def camera_extract(row):
    row_splitted = row.split()
    pattern = r'\d+MP'
    words = ' '.join(row_splitted)
    find_cam = re.findall(pattern,words)
    return ' '.join(find_cam)


In [50]:
df['camera'] = df['camera'].apply(camera_extract)

In [51]:
def camera(txt):
    if not txt:
        return 13

    lst = []
    splitted = txt.split()
    for i in splitted:
        make_int = i.replace('MP', '')
        lst.append(int(make_int))

    return max(lst)


In [52]:
random_values('camera')

59 : 8MP 2MP
2 : 50MP 8MP 2MP 50MP 8MP 2MP 16MP 16MP
83 : 50MP 50MP 2MP 50MP 50MP 2MP 60MP 60MP
64 : 16MP 2MP 2MP 16MP 2MP 2MP 8MP 8MP
89 : 50MP 8MP 2MP 50MP 8MP 2MP 50MP 16MP 16MP
81 : 108MP 8MP 16MP 108MP 8MP 16MP 9MP 32MP 32MP


In [53]:
df['camera'] = df['camera'].apply(func=camera)
df['camera'] = df['camera'].astype('Int16')

## display

In [54]:
def display_extract(row):
    split_row = row.split()
    if 'LCD' in split_row:
        if 'IPS' in split_row and 'LCD' in split_row:
            return 'IPS LCD'
        return 'LCD'
    
    if 'OLED' in split_row:
        return 'OLED'
    
    if 'TFT' in split_row:
        return 'TFT'
    
    if 'LED' in split_row:
        return 'LED'

    # if 'Adreno' in split_row:
    #     return 'Adreno'
    if 'AMOLED' in split_row:
        if 'HD+' in split_row and 'AMOLED' in split_row:
            return 'HD+ AMOLED'
        return 'AMOLED'
    
    return 'other'


In [55]:
df['display'] = df['display'].apply(display_extract)

## network

In [57]:
df = df.drop(columns=['network'])

## battery

In [58]:
def extract_battery(row):
    return row.split()[1]

df['battery'] = df['battery'].apply(extract_battery)

In [59]:
df.reset_index(inplace=True)
count = 0
for i in range(0,len(df['battery'])):
    # print(i)
    df['battery'][i]= df['battery'].iloc[i].replace('Capacity','')
    # print(df['battery'][i])
    count+=1
print(f'looped till {count} index')

looped till 110 index


In [60]:
df['battery'] = df['battery'].astype('Int16')

In [61]:
df = df.drop(labels='index',axis=1)

In [64]:
df

,name,ratings,price,imgURL,camera,display,battery,ram,storage,android_version,processor
0,"MOTOROLA g42 (Atlantic Green, 64 GB)",4.2,9999,https://rukminim2.flixcart.com/image/312/312/l...,50,HD+ AMOLED,5000,4,64,12,Snapdragon680P
1,"MOTOROLA e13 (Cosmic Black, 64 GB)",4.0,7299,https://rukminim2.flixcart.com/image/312/312/x...,13,IPS LCD,5000,4,64,606,UnisocT606Processor
2,"MOTOROLA G32 (Mineral Gray, 128 GB)",4.1,11999,https://rukminim2.flixcart.com/image/312/312/x...,50,IPS LCD,5000,8,128,12,Snapdragon680P
3,"MOTOROLA e32 (Arctic Blue, 64 GB)",3.9,7999,https://rukminim2.flixcart.com/image/312/312/x...,50,IPS LCD,5000,4,64,12,Heilo37P23184135782038404131582358
4,"MOTOROLA g13 (Lavender Blue, 128 GB)",3.9,9999,https://rukminim2.flixcart.com/image/312/312/x...,50,IPS LCD,5000,4,128,13,Heilo85P24123578203840413125822358
...,...,...,...,...,...,...,...,...,...,...,...
105,"Moto E6s (Polished Graphite, 64 GB)",4.1,7499,https://rukminim2.flixcart.com/image/312/312/k...,13,other,3000,4,64,11,Snapdragon662P
106,"MOTOROLA G30 (Pastel Sky, 64 GB)",4.1,10999,https://rukminim2.flixcart.com/image/312/312/k...,64,IPS LCD,5000,4,64,11,Snapdragon460P
107,"MOTOROLA G10 Power (Breeze Blue, 64 GB)",4.2,9999,https://rukminim2.flixcart.com/image/312/312/k...,48,IPS LCD,6000,4,64,11,Dimensity800UProcessor
108,"MOTOROLA Edge 20 Fusion 5G (Cyber Teal, 128 GB)",4.0,18999,https://rukminim2.flixcart.com/image/312/312/k...,108,HD+ AMOLED,5000,6,128,12,Gen8


In [65]:
df.to_csv(r'data\cleaned_dataset\motorola_plus_cleaned_data.csv')